<a href="https://colab.research.google.com/github/rjarun8/Deep_Learning/blob/master/keras_hyper_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Packages

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

#Keras/TF
import tensorflow
from tensorflow import keras
from keras.layers.core import Dense, Dropout, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from keras.utils import np_utils
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical


#NLP Packages
import spacy
from string import punctuation
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
punctuations = string.punctuation
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg') ## Spacy english large model
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

#Hyper Tuning
#pip install keras-tuner
from kerastuner.tuners import Hyperband
from kerastuner import HyperModel

#Data Pre-processing and Preparing train & Test Data

In [22]:
#read from file
dfnew=pd.read_excel(r'/content/netcool_inc_dump.xlsx')

#taking required feature and label
df=dfnew[['Assigned Group','Summary']]

#convert column to lower
df.columns = map(str.lower, df.columns)

#rename column
df.rename(columns={"assigned group": "assigned_group",}, inplace=True)

#convert label  to lower
df['assigned_group'] = df['assigned_group'].apply(lambda x: x.lower())

#boosting label values
list_data = ['online operations itcc services','wholesale operations',
            'mobile number reuse support mnr','professional services',
            'external mipay it infrastructure use only' , 
            'external hp it infrastructure use only', 
            'external experian it infrastructure use only' ,  
            'external equifax it infrastructure use only','ibm power systems operations l1','ibm power systems support l2','external equifax (it infrastructure use only)',
             'external experian (it infrastructure use only)',
             'external hp (it infrastructure use only)',
             'external mipay (it infrastructure use only)','mobile number reuse support (mnr)','online operations (itcc services)']
for i in range(len(df['assigned_group'])):
    for j in range(len(list_data)):
        if list_data[j] in df['assigned_group'][i]:
            df['assigned_group'][i]='others'

#assigning label to y
y = df['assigned_group']

#assignping feature to X
X=df['summary']

#using Spacy for NLP pre-processing - Tokenize, Lemmatize, stop words
def spacy_tokenizer(sentence):
  # Creating our token object, which is used to create documents with linguistic annotations.
  mytokens = parser(sentence)

  # Lemmatizing each token and converting each token into lowercase
  mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

  # Removing stop words
  mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
  # return preprocessed list of tokens
  return mytokens

#vectorizing inpur feature
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer,max_features=10000)
y = np_utils.to_categorical(df['assigned_group'].factorize()[0], df['assigned_group'].nunique())
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size = 0.1,random_state=42)
tfidf_vector.fit(x_train)
X_trV=tfidf_vector.transform(x_train)
X_tsV=tfidf_vector.transform(x_test)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

#Creating Hyper Model

In [0]:
class MyHyperModel(HyperModel):
  def __init__(self, num_classes):
    self.num_classes = num_classes
  def build(self, hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 5)):
      model.add(Dropout(0.2))
      model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),min_value=8,max_value=512, step=8),activation='relu'))        
        # Output layer
    model.add(keras.layers.Dense(self.num_classes, activation='softmax'))
        
        # Compile the constructed model and return it
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),loss='categorical_crossentropy', metrics=['accuracy'])

    return model
    

#Model Initialization

In [0]:
hypermodel = MyHyperModel(num_classes=len(y[1]))

#Constructing Hyperband tuner

In [0]:
tuner = Hyperband(
    hypermodel,
    objective='val_acc',
    max_epochs=20,
    seed=10,
    project_name='Keras_hyperas')

#Search for the best parameters of the neural network using the contructed Hypberband tuner

In [30]:
tuner.search(X_trV.toarray(), y_train, epochs=10, validation_data=(X_tsV.toarray(), y_test))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 8644 samples, validate on 961 samples
Epoch 1/3
8644/8644 [==============================] - 6s 689us/sample - loss: 2.0382 - acc: 0.2846 - val_loss: 1.6299 - val_acc: 0.4527
Epoch 2/3
8644/8644 [==============================] - 6s 647us/sample - loss: 1.4932 - acc: 0.4722 - val_loss: 1.2472 - val_acc: 0.5734
Epoch 3/3
8644/8644 [==============================] - 6s 644us/sample - loss: 1.2301 - acc: 0.5650 - val_loss: 1.1514 - val_acc: 0.6504


Train on 8644 samples, validate on 961 samples
Epoch 1/3
8644/8644 [==============================] - 9s 1ms/sample - loss: 2.2831 - acc: 0.2029 - val_loss: 2.0900 - val_acc: 0.2300
Epoch 2/3
8644/8644 [==============================] - 9s 1ms/sample - loss: 1.7944 - acc: 0.4057 - val_loss: 1.4459 - val_acc: 0.5775
Epoch 3/3
8644/8644 [==============================] - 9s 1ms/sample - loss: 1.2427 - acc: 0.6481 - val_loss: 1.0754 - val_acc: 0.7055


Train on 8644 samples, validate on 961 samples
Epoch 1/3
8644/8644 [==============================] - 5s 596us/sample - loss: 2.0697 - acc: 0.2141 - val_loss: 1.5802 - val_acc: 0.4891
Epoch 2/3
8644/8644 [==============================] - 5s 586us/sample - loss: 1.2523 - acc: 0.6038 - val_loss: 1.0609 - val_acc: 0.6795
Epoch 3/3
8644/8644 [==============================] - 5s 591us/sample - loss: 0.8809 - acc: 0.7295 - val_loss: 0.9605 - val_acc: 0.7274


Train on 8644 samples, validate on 961 samples
Epoch 1/3
8644/8644 [==============================] - 8s 918us/sample - loss: 2.3365 - acc: 0.1948 - val_loss: 2.2205 - val_acc: 0.1935
Epoch 2/3
8644/8644 [==============================] - 8s 877us/sample - loss: 2.0348 - acc: 0.2525 - val_loss: 1.7900 - val_acc: 0.3913
Epoch 3/3
8644/8644 [==============================] - 8s 882us/sample - loss: 1.6182 - acc: 0.4488 - val_loss: 1.4136 - val_acc: 0.5213


Train on 8644 samples, validate on 961 samples
Epoch 1/3
8644/8644 [==============================] - 8s 953us/sample - loss: 1.4834 - acc: 0.5079 - val_loss: 0.9097 - val_acc: 0.7097
Epoch 2/3
8644/8644 [==============================] - 8s 917us/sample - loss: 0.7787 - acc: 0.7575 - val_loss: 0.8180 - val_acc: 0.7659
Epoch 3/3
8644/8644 [==============================] - 8s 900us/sample - loss: 0.5462 - acc: 0.8313 - val_loss: 0.8231 - val_acc: 0.7575


...

Train on 8644 samples, validate on 961 samples
Epoch 1/20
8644/8644 [==============================] - 6s 670us/sample - loss: 1.4721 - acc: 0.5177 - val_loss: 0.8582 - val_acc: 0.7263
Epoch 2/20
8644/8644 [==============================] - 6s 666us/sample - loss: 0.7084 - acc: 0.7790 - val_loss: 0.7243 - val_acc: 0.7700
Epoch 3/20
8644/8644 [==============================] - 6s 660us/sample - loss: 0.5003 - acc: 0.8390 - val_loss: 0.7819 - val_acc: 0.7617
Epoch 4/20
8644/8644 [==============================] - 6s 657us/sample - loss: 0.4239 - acc: 0.8616 - val_loss: 0.7727 - val_acc: 0.7763
Epoch 5/20
8644/8644 [==============================] - 6s 644us/sample - loss: 0.3829 - acc: 0.8707 - val_loss: 0.8263 - val_acc: 0.7617
Epoch 6/20
8644/8644 [==============================] - 6s 656us/sample - loss: 0.3626 - acc: 0.8760 - val_loss: 0.8276 - val_acc: 0.7794
Epoch 7/20
8644/8644 [==============================] - 6s 688us/sample - loss: 0.3534 - acc: 0.8780 - val_loss: 0.8399 - val

Train on 8644 samples, validate on 961 samples
Epoch 1/20
8644/8644 [==============================] - 3s 374us/sample - loss: 2.3459 - acc: 0.1988 - val_loss: 2.2842 - val_acc: 0.1935
Epoch 2/20
8644/8644 [==============================] - 3s 357us/sample - loss: 2.1926 - acc: 0.2264 - val_loss: 2.0259 - val_acc: 0.2893
Epoch 3/20
8644/8644 [==============================] - 3s 361us/sample - loss: 1.9169 - acc: 0.3326 - val_loss: 1.7116 - val_acc: 0.4100
Epoch 4/20
8644/8644 [==============================] - 3s 378us/sample - loss: 1.6954 - acc: 0.4143 - val_loss: 1.5197 - val_acc: 0.4932
Epoch 5/20
8644/8644 [==============================] - 3s 393us/sample - loss: 1.5642 - acc: 0.4434 - val_loss: 1.3987 - val_acc: 0.5297
Epoch 6/20
8644/8644 [==============================] - 3s 390us/sample - loss: 1.4584 - acc: 0.4942 - val_loss: 1.3143 - val_acc: 0.5973
Epoch 7/20
8644/8644 [==============================] - 3s 387us/sample - loss: 1.3941 - acc: 0.5207 - val_loss: 1.2482 - val

Train on 8644 samples, validate on 961 samples
Epoch 1/20
8644/8644 [==============================] - 5s 532us/sample - loss: 1.9918 - acc: 0.2805 - val_loss: 1.4818 - val_acc: 0.4995
Epoch 2/20
8644/8644 [==============================] - 4s 516us/sample - loss: 1.2440 - acc: 0.5772 - val_loss: 0.9918 - val_acc: 0.6982
Epoch 3/20
8644/8644 [==============================] - 5s 540us/sample - loss: 0.8892 - acc: 0.7154 - val_loss: 0.8964 - val_acc: 0.7336
Epoch 4/20
8644/8644 [==============================] - 5s 531us/sample - loss: 0.7218 - acc: 0.7778 - val_loss: 0.8650 - val_acc: 0.7482
Epoch 5/20
8644/8644 [==============================] - 4s 518us/sample - loss: 0.6341 - acc: 0.8063 - val_loss: 0.8927 - val_acc: 0.7659
Epoch 6/20
8644/8644 [==============================] - 4s 505us/sample - loss: 0.5714 - acc: 0.8254 - val_loss: 0.8677 - val_acc: 0.7627
Epoch 7/20
8644/8644 [==============================] - 4s 515us/sample - loss: 0.5408 - acc: 0.8331 - val_loss: 0.8476 - val

Train on 8644 samples, validate on 961 samples
Epoch 1/20
8644/8644 [==============================] - 10s 1ms/sample - loss: 2.2776 - acc: 0.1952 - val_loss: 2.0575 - val_acc: 0.2664
Epoch 2/20
8644/8644 [==============================] - 10s 1ms/sample - loss: 1.6731 - acc: 0.4342 - val_loss: 1.3209 - val_acc: 0.5609
Epoch 3/20
8644/8644 [==============================] - 10s 1ms/sample - loss: 1.1724 - acc: 0.6165 - val_loss: 1.0432 - val_acc: 0.6878
Epoch 4/20
8644/8644 [==============================] - 10s 1ms/sample - loss: 0.9291 - acc: 0.7173 - val_loss: 0.9239 - val_acc: 0.7305
Epoch 5/20
8644/8644 [==============================] - 10s 1ms/sample - loss: 0.7706 - acc: 0.7638 - val_loss: 0.8844 - val_acc: 0.7544
Epoch 6/20
8644/8644 [==============================] - 10s 1ms/sample - loss: 0.6629 - acc: 0.8010 - val_loss: 0.8702 - val_acc: 0.7503
Epoch 7/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.5890 - acc: 0.8228 - val_loss: 0.8350 - val_acc: 0.

INFO:tensorflow:Oracle triggered exit


#Get Best Hyper Params

In [0]:
params = tuner.get_best_hyperparameters()[0]

#Build  The model with the best params

In [34]:
model = tuner.hypermodel.build(params)
model.fit(X_trV.toarray(), y_train, epochs=20)
model.save('keras_hypeband_best_param_model.hd5')

Train on 8644 samples
Epoch 1/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 1.4029 - acc: 0.5419
Epoch 2/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.6713 - acc: 0.7879
Epoch 3/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.4769 - acc: 0.8460
Epoch 4/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.4199 - acc: 0.8623
Epoch 5/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.3794 - acc: 0.8744
Epoch 6/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.3507 - acc: 0.8797
Epoch 7/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.3434 - acc: 0.8830
Epoch 8/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.3244 - acc: 0.8864
Epoch 9/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.3202 - acc: 0.8852
Epoch 10/20
8644/8644 [==============================] - 9s 1ms/sample - loss: 0.3062 -

#Hyper model accuracy

In [36]:
hyperband_accuracy_df = pd.DataFrame(model.history.history)
hyperband_accuracy_df

,loss,acc
0,1.402917,0.541879
1,0.671324,0.787945
2,0.476888,0.846020
3,0.419939,0.862332
4,0.379356,0.874364
5,0.350729,0.879685
6,0.343392,0.883040
7,0.324375,0.886395
8,0.320167,0.885238
9,0.306207,0.888940


#testing hyper model  accuracy

In [37]:
loss, accuracy = model.evaluate(X_trV.toarray(), y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_tsV.toarray(), y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9447
Testing Accuracy:  0.7659


#Generate Classification Report

In [38]:
from sklearn.metrics import confusion_matrix,classification_report
predictions = model.predict_classes(X_tsV.toarray())
print(classification_report(y_test.argmax(axis=1),predictions))

              precision    recall  f1-score   support

           0       0.77      0.75      0.76        55
           1       0.75      0.78      0.77        90
           2       0.86      0.80      0.83       186
           3       0.64      0.74      0.69        81
           4       0.82      0.94      0.88        70
           5       0.83      0.59      0.69       128
           6       0.73      0.77      0.75        82
           7       0.70      0.66      0.68        64
           8       0.63      0.76      0.69        84
           9       0.89      0.92      0.91        53
          10       0.76      0.82      0.79        68

    accuracy                           0.77       961
   macro avg       0.76      0.78      0.77       961
weighted avg       0.77      0.77      0.77       961



#Retrain the hyper model with the data set

In [40]:
callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_trV.toarray(), y_train, epochs=100, verbose=2, validation_data=(X_tsV.toarray(), y_test), batch_size=20,callbacks=[callback])
model.save('keras_hypeband_best_param_model_callback.hd5')

Train on 8644 samples, validate on 961 samples
Epoch 1/100
8644/8644 - 10s - loss: 0.2945 - acc: 0.8894 - val_loss: 0.9563 - val_acc: 0.7627
Epoch 2/100
8644/8644 - 10s - loss: 0.2802 - acc: 0.8952 - val_loss: 0.9310 - val_acc: 0.7596
Epoch 3/100
8644/8644 - 10s - loss: 0.2739 - acc: 0.8973 - val_loss: 1.1010 - val_acc: 0.7815
Epoch 4/100
8644/8644 - 10s - loss: 0.2769 - acc: 0.8951 - val_loss: 1.0036 - val_acc: 0.7732
Epoch 5/100
8644/8644 - 10s - loss: 0.2734 - acc: 0.9020 - val_loss: 1.0392 - val_acc: 0.7752


#Check train and test accuracy

In [41]:
loss, accuracy = model.evaluate(X_trV, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_tsV, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9441
Testing Accuracy:  0.7752


#Generate Confusion Matrix

In [42]:
from sklearn.metrics import confusion_matrix,classification_report
predictions = model.predict_classes(X_tsV.toarray())
print(classification_report(y_test.argmax(axis=1),predictions))

              precision    recall  f1-score   support

           0       0.77      0.73      0.75        55
           1       0.76      0.74      0.75        90
           2       0.84      0.83      0.83       186
           3       0.72      0.68      0.70        81
           4       0.86      0.90      0.88        70
           5       0.78      0.69      0.73       128
           6       0.83      0.72      0.77        82
           7       0.69      0.73      0.71        64
           8       0.61      0.80      0.69        84
           9       0.83      0.92      0.88        53
          10       0.82      0.82      0.82        68

    accuracy                           0.78       961
   macro avg       0.77      0.78      0.77       961
weighted avg       0.78      0.78      0.78       961

